In [1]:
!pip install opencv-python matplotlib numpy pandas csv

ERROR: Could not find a version that satisfies the requirement csv (from versions: none)
ERROR: No matching distribution found for csv

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import csv

In [3]:
# Complete the Incomplete Curves
def read_csv(csv_path):
    data = pd.read_csv(csv_path, header=None)
    shapes = []
    for shape_id in data[0].unique():
        shape_data = data[data[0] == shape_id]
        paths = []
        for path_id in shape_data[1].unique():
            path_data = shape_data[shape_data[1] == path_id].iloc[:, 2:].to_numpy()
            paths.append(path_data)
        shapes.append(paths)
    return shapes

def close_curve(path):
    if not np.array_equal(path[0], path[-1]):
        path = np.vstack([path, path[0]])  # Join the endpoints
    return path

def measure_and_classify_curve(contour):
    shape_type = 'Unknown Curve'
    center = (0, 0)
    axes = (0, 0)
    angle = 0
    if len(contour) >= 5:
        ellipse = cv2.fitEllipse(contour)
        center, axes, angle = ellipse
        shape_type = 'Ellipse'
    elif len(contour) >= 3:
        # Check if the curve is a circle by comparing axes lengths
        ellipse = cv2.fitEllipse(contour)
        axes = ellipse[1]
        if abs(axes[0] - axes[1]) < 0.1 * min(axes):
            shape_type = 'Circle'
            center, axes, angle = ellipse

    return shape_type, center, axes, angle

def draw_incomplete_parts(contour, img):
    # Calculate distance between first and last point
    start_point = tuple(contour[0][0])
    end_point = tuple(contour[-1][0])
    cv2.line(img, start_point, end_point, (0, 0, 255), 2)

def detect_and_draw_curves(shapes, output_csv):
    detected_shapes = []

    # Create a blank image for visualization
    img = np.zeros((500, 500, 3), dtype=np.uint8) + 255

    for shape in shapes:
        for path in shape:
            contour = np.array(path, dtype=np.int32).reshape((-1, 1, 2))
            shape_type, center, axes, angle = measure_and_classify_curve(contour)
            
            if shape_type == 'Ellipse' or shape_type == 'Circle':
                cv2.ellipse(img, (center, axes, angle), (0, 255, 0), 2)
                detected_shapes.append([shape_type, center[0], center[1], axes[0], axes[1], angle])
            else:
                closed_contour = close_curve(path)
                closed_contour = np.array(closed_contour, dtype=np.int32).reshape((-1, 1, 2))
                if not np.array_equal(contour, closed_contour):
                    draw_incomplete_parts(contour, img)
                    detected_shapes.append(['Incomplete Curve', contour[0][0][0], contour[0][0][1], contour[-1][0][0], contour[-1][0][1]])
                else:
                    cv2.drawContours(img, [closed_contour], 0, (0, 0, 255), 2)
                    M = cv2.moments(contour)
                    if M['m00'] != 0.0:
                        x = int(M['m10'] / M['m00'])
                        y = int(M['m01'] / M['m00'])
                        detected_shapes.append([shape_type, x, y, cv2.arcLength(contour, True), cv2.contourArea(contour)])

    # Save detected shapes to CSV
    with open(output_csv, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Shape', 'Center_X', 'Center_Y', 'Axis1', 'Axis2', 'Angle'])
        writer.writerows(detected_shapes)

    # Show the image with detected shapes
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.show()

csv_path = "problems/occlusion1.csv"  # Replace with your CSV file path
output_csv = 'output/detected_shapes.csv'  # Replace with desired output CSV file path
shapes = read_csv(csv_path)
detect_and_draw_curves(shapes, output_csv)

FileNotFoundError: [Errno 2] No such file or directory: 'problems/occlusion1.csv'

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

PIXELS_TO_MM = 0.2645833333

def read_csv(csv_path):
    np_path_XYs = np.genfromtxt(csv_path, delimiter=',')
    path_XYs = []
    for i in np.unique(np_path_XYs[:, 0]):
        npXYs = np_path_XYs[np_path_XYs[:, 0] == i][:, 1:]
        XYs = []
        for j in np.unique(npXYs[:, 0]):
            XY = npXYs[npXYs[:, 0] == j][:, 1:]
            XYs.append(XY)
        path_XYs.append(XYs)
    return path_XYs

def draw_shapes_on_blank_image(paths_XYs, image_size):
    img = np.ones(image_size, dtype=np.uint8) * 255  # Create a white blank image in grayscale
    for XYs in paths_XYs:
        for XY in XYs:
            pts = XY.reshape((-1, 1, 2)).astype(np.int32)
            cv2.polylines(img, [pts], isClosed=False, color=0, thickness=2)  # Draw in black
    return img

def get_contours(img, imgContour, detected_shapes):
    contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        area_og = int(cv2.contourArea(cnt))
        if area_og > 50:
            epsilon = 0.02 * cv2.arcLength(cnt, True)
            approx_points = cv2.approxPolyDP(cnt, epsilon, True)
            object_corners = len(approx_points)
            x, y, w, h = cv2.boundingRect(approx_points)

            # Check if the contour is a circle
            (x_center, y_center), radius = cv2.minEnclosingCircle(cnt)
            radius = int(radius)
            circle_area = np.pi * (radius ** 2)
            area_diff = abs(area_og - circle_area) / circle_area
            
            if area_diff < 0.2:  # It's a circle if the contour area is close to the circle area
                objectType = "Circle"
                detected_shapes.append({
                    'type': 'Circle',
                    'center': (int(x_center), int(y_center)),
                    'radius': radius
                })
                cv2.circle(imgContour, (int(x_center), int(y_center)), radius, 0, 2)  # Draw circle in black
            else:
                # Determine shape based on vertices
                if object_corners > 7:
                    objectType = "Star"
                elif object_corners == 6:
                    objectType = "Hexagon"
                elif object_corners == 4:
                    aspRatio = w / float(h)
                    if 0.9 < aspRatio < 1.2:
                        objectType = "Square"
                    else:
                        objectType = "Rectangle"
                elif object_corners == 3:
                    objectType = "Triangle"
                else:
                    objectType = "Polygon"

                area = int(cv2.contourArea(cnt) * PIXELS_TO_MM)
                perimeter = int(cv2.arcLength(cnt, True) * PIXELS_TO_MM)

                cv2.polylines(imgContour, [approx_points], isClosed=True, color=0, thickness=2)  # Draw in black
                cv2.putText(imgContour, objectType, (x + w // 2 - 20, y + h // 2 - 14),
                            cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.51, 0, 1)  # Text in black
                cv2.putText(imgContour, "A: " + str(area) + "mm", (x + w // 2 - 30, y + h // 2),
                            cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.5, 0, 1)  # Text in black
                cv2.putText(imgContour, "P: " + str(perimeter) + "mm", (x + w // 2 - 30, y + h // 2 + 12),
                            cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.5, 0, 1)  # Text in black

                detected_shapes.append({
                    'type': objectType,
                    'vertices': approx_points.reshape(-1, 2).tolist(),
                    'bounding_rect': (x, y, w, h),
                    'area': area,
                    'perimeter': perimeter
                })

: 

In [ ]:
# Detect and Regularize isolated figures
def draw_detected_shapes(detected_shapes, image_size):
    img = np.ones(image_size, dtype=np.uint8) * 255  # Create a white blank image in grayscale
    for shape in detected_shapes:
        if shape['type'] == 'Circle':
            center, radius = shape['center'], shape['radius']
            cv2.circle(img, center, radius, 0, 2)  # Draw circle in black
        elif shape['type'] == 'Rectangle' or shape['type'] == 'Square':
            x, y, w, h = shape['bounding_rect']
            cv2.rectangle(img, (x, y), (x + w, y + h), 0, 2)  # Draw rectangle in black
        elif shape['type'] == 'Triangle':
            pts = np.array(shape['vertices'], np.int32)
            cv2.polylines(img, [pts.reshape((-1, 1, 2))], isClosed=True, color=0, thickness=2)
        elif shape['type'] == 'Polygon' or shape['type'] == 'Star':
            pts = np.array(shape['vertices'], np.int32)
            cv2.polylines(img, [pts.reshape((-1, 1, 2))], isClosed=True, color=0, thickness=2)
    return img

# Define the path to the CSV file
csv_path = 'problems/isolated.csv'

# Read the CSV file
paths_XYs = read_csv(csv_path)

# Draw the shapes on a blank image
image_size = (800, 800)  # Define the size of the blank image in grayscale
img = draw_shapes_on_blank_image(paths_XYs, image_size)
imgContour = img.copy()

# Convert to grayscale and apply GaussianBlur and Canny edge detector
imgGray = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)  # Convert grayscale image to BGR for Canny
imgBlur = cv2.GaussianBlur(imgGray, (7, 7), 1)
imgCanny = cv2.Canny(imgBlur, 50, 150)

# Get contours and annotate the image
detected_shapes = []
get_contours(imgCanny, imgContour, detected_shapes)

# Draw a new image with detected shapes
detected_shapes_image = draw_detected_shapes(detected_shapes, image_size)

# Save and show the new image with detected shapes
cv2.imwrite('output/detected_shapes_image.jpg', detected_shapes_image)
cv2.imshow("Detected Shapes", detected_shapes_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

: 

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

PIXELS_TO_MM = 0.2645833333

def read_csv(csv_path):
    np_path_XYs = np.genfromtxt(csv_path, delimiter=',')
    path_XYs = []
    for i in np.unique(np_path_XYs[:, 0]):
        npXYs = np_path_XYs[np_path_XYs[:, 0] == i][:, 1:]
        XYs = []
        for j in np.unique(npXYs[:, 0]):
            XY = npXYs[npXYs[:, 0] == j][:, 1:]
            XYs.append(XY)
        path_XYs.append(XYs)
    return path_XYs

def draw_shapes_on_blank_image(paths_XYs, image_size):
    img = np.ones(image_size, dtype=np.uint8) * 255  # Create a white blank image in grayscale
    for XYs in paths_XYs:
        for XY in XYs:
            pts = XY.reshape((-1, 1, 2)).astype(np.int32)
            cv2.polylines(img, [pts], isClosed=False, color=0, thickness=2)  # Draw in black
    return img

def process_contour(cnt, imgContour, detected_shapes, hierarchy, index):
    area_og = int(cv2.contourArea(cnt))
    if area_og > 50:
        epsilon = 0.02 * cv2.arcLength(cnt, True)
        approx_points = cv2.approxPolyDP(cnt, epsilon, True)
        object_corners = len(approx_points)
        x, y, w, h = cv2.boundingRect(approx_points)

        # Check if the contour is a circle
        (x_center, y_center), radius = cv2.minEnclosingCircle(cnt)
        radius = int(radius)
        circle_area = np.pi * (radius ** 2)
        area_diff = abs(area_og - circle_area) / circle_area
        
        if area_diff < 0.2:  # It's a circle if the contour area is close to the circle area
            objectType = "Circle"
            detected_shapes.append({
                'type': 'Circle',
                'center': (int(x_center), int(y_center)),
                'radius': radius,
                'hierarchy': hierarchy[index].tolist()
            })
            cv2.circle(imgContour, (int(x_center), int(y_center)), radius, 0, 2)  # Draw circle in black
        else:
            # Determine shape based on vertices
            if object_corners > 7:
                objectType = "Star"
            elif object_corners == 6:
                objectType = "Hexagon"
            elif object_corners == 4:
                aspRatio = w / float(h)
                if 0.9 < aspRatio < 1.2:
                    objectType = "Square"
                else:
                    objectType = "Rectangle"
            elif object_corners == 3:
                objectType = "Triangle"
            else:
                objectType = "Polygon"

            area = int(cv2.contourArea(cnt) * PIXELS_TO_MM)
            perimeter = int(cv2.arcLength(cnt, True) * PIXELS_TO_MM)

            cv2.polylines(imgContour, [approx_points], isClosed=True, color=0, thickness=2)  # Draw in black
            cv2.putText(imgContour, objectType, (x + w // 2 - 20, y + h // 2 - 14),
                        cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.51, 0, 1)  # Text in black
            cv2.putText(imgContour, "A: " + str(area) + "mm", (x + w // 2 - 30, y + h // 2),
                        cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.5, 0, 1)  # Text in black
            cv2.putText(imgContour, "P: " + str(perimeter) + "mm", (x + w // 2 - 30, y + h // 2 + 12),
                        cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.5, 0, 1)  # Text in black

            detected_shapes.append({
                'type': objectType,
                'vertices': approx_points.reshape(-1, 2).tolist(),
                'bounding_rect': (x, y, w, h),
                'area': area,
                'perimeter': perimeter,
                'hierarchy': hierarchy[index].tolist()
            })

def get_contours(img, imgContour, detected_shapes):
    contours, hierarchy = cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    hierarchy = hierarchy[0]  # Get the actual hierarchy list
    for i, cnt in enumerate(contours):
        process_contour(cnt, imgContour, detected_shapes, hierarchy, i)

: 

In [ ]:
# Draw Detected shapes
def draw_detected_shapes(detected_shapes, image_size):
    img = np.ones(image_size, dtype=np.uint8) * 255  # Create a white blank image in grayscale
    for shape in detected_shapes:
        if shape['type'] == 'Circle':
            center, radius = shape['center'], shape['radius']
            cv2.circle(img, center, radius, 0, 2)  # Draw circle in black
        elif shape['type'] == 'Rectangle' or shape['type'] == 'Square':
            x, y, w, h = shape['bounding_rect']
            cv2.rectangle(img, (x, y), (x + w, y + h), 0, 2)  # Draw rectangle in black
        elif shape['type'] == 'Triangle':
            pts = np.array(shape['vertices'], np.int32)
            cv2.polylines(img, [pts.reshape((-1, 1, 2))], isClosed=True, color=0, thickness=2)
        elif shape['type'] == 'Polygon' or shape['type'] == 'Star':
            pts = np.array(shape['vertices'], np.int32)
            cv2.polylines(img, [pts.reshape((-1, 1, 2))], isClosed=True, color=0, thickness=2)
    return img

# Define the path to the CSV file
csv_path = 'problems/frag0.csv'

# Read the CSV file
paths_XYs = read_csv(csv_path)

# Draw the shapes on a blank image
image_size = (800, 800)  # Define the size of the blank image in grayscale
img = draw_shapes_on_blank_image(paths_XYs, image_size)
imgContour = img.copy()

# Convert to grayscale and apply GaussianBlur and Canny edge detector
imgGray = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)  # Convert grayscale image to BGR for Canny
imgBlur = cv2.GaussianBlur(imgGray, (7, 7), 1)
imgCanny = cv2.Canny(imgBlur, 50, 150)

# Get contours and annotate the image
detected_shapes = []
get_contours(imgCanny, imgContour, detected_shapes)

# Draw a new image with detected shapes
detected_shapes_image = draw_detected_shapes(detected_shapes, image_size)

# Save and show the new image with detected shapes
cv2.imwrite('output/detected_shapes_image.jpg', detected_shapes_image)
cv2.imshow("Detected Shapes", detected_shapes_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

: 